In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
predictors = pd.read_csv('C:/Users/patel/Thesis/DataCleaning/Food/RF/inputdata.csv')
predicted = pd.read_csv('Meat_consume.csv')
#predicted = predicted[predicted['UNIT']=='Thousand tonnes']
predicted = predicted.dropna()

In [23]:
predicted = predicted[['TIME','GEO','Meat','Value']]
typeof = 'Meat'
predicted_reformat = pd.DataFrame()
number_of_col = 1
for type_unique in predicted[typeof].unique():
    if (number_of_col == 1):
        predicted_unique = predicted[predicted[typeof]==type_unique]
        predicted_reformat['TIME'] = predicted_unique['TIME']
        predicted_reformat['GEO'] = predicted_unique['GEO']
        predicted_unique['Value'].replace({":":0}, inplace=True) 
        predicted_reformat[type_unique] = predicted_unique['Value'].replace({' ':''},regex=True).astype('float')
        predicted_reformat = predicted_reformat[predicted_reformat[type_unique] != 0]
        number_of_col = number_of_col + 1
    else:
        predicted_unique = predicted[predicted[typeof]==type_unique]
        predicted_unique['Value'].replace({":":0}, inplace=True) 
        predicted_unique[type_unique] = predicted_unique['Value'].replace({' ':''},regex=True).astype('float')
        predicted_unique = predicted_unique[predicted_unique[type_unique] != 0]
        predicted_reformat = predicted_reformat.merge(predicted_unique[['TIME','GEO',type_unique]],on=['TIME','GEO'],how='inner')
predicted_reformat['Total_consumed'] = predicted_reformat.iloc[:,2:].sum(axis=1)
predicted_reformat = predicted_reformat[['TIME','GEO','Total_consumed']]
predicted_reformat=predicted_reformat[((predicted_reformat['Total_consumed'] - predicted_reformat['Total_consumed'].mean()) / predicted_reformat['Total_consumed'].std()).abs() < 2]
input_output = pd.merge(predictors,predicted_reformat,on=['TIME','GEO'],how='inner')


C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
predicted_unique['Value'].unique()

array(['1 712.00', '996 100.00', '23.39', '15 101.07', '1 342.32',
       '683 701.56', '122 707.00', '1 733.00', '1 030 212.00', '22.81',
       '14 640.89', '108 100.63', '147.2', '1 384.24', '703 865.35',
       '1 709.00', '1 005 223.00', '24.49', '15 154.94', '104 767.01',
       '1 695.00', '1 016 708.00', '55.7', '105 402.98', '1 678.00',
       '972 890.00', '59.1', '35 731.00', '113 556.24', '1 719.00',
       '1 006 004.00', '63.4', '37 969.90', '58.76', '33 321.69',
       '115 860.94', '64', '38 205.60', '605 011.00', '1 791.26',
       '1 077 017.28', '117 922.55', '1 528.85', '754 383.65', '1 650.00',
       '955 350.00', '64.5', '38 447.60', '121 008.16', '1 732.00',
       '974 045.00', '452.5', '264 575.65', '123 520.51', '1 736.12',
       '828 047.54', '1 698.00', '940 369.00', '482.28', '280 546.44'],
      dtype=object)

In [25]:
input_output['Per capita consumption'] = input_output['Total_consumed']*1000000/input_output['Total_pop']
final_input_output = input_output.drop(['Total_pop','Total_consumed'],axis=1)
final_input_output=final_input_output[((final_input_output['Per capita consumption'] - final_input_output['Per capita consumption'].mean()) / final_input_output['Per capita consumption'].std()).abs() < 2]
final_input_output.to_csv('final_input_output.csv',index=False)